In [2]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

In [3]:
# open the file in Instances folder
f = open("inst05.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 2
items: 3
load_size: [18, 30]
item_size: [20, 17, 6]
distance:
 [[ 0 21 86 99]
 [21  0 71 80]
 [92 71  0 61]
 [59 80 61  0]]


In [4]:
# implement the graph based approach as an adjacency matrix m x n x n
graph_nodes = [[[Bool("x_%s_%s_%s" % (i, j, k)) for k in range(m)] for j in range(n + 1)] for i in range(n + 1)]
print("graph_nodes:\n", graph_nodes)

graph_nodes:
 [[[x_0_0_0, x_0_0_1], [x_0_1_0, x_0_1_1], [x_0_2_0, x_0_2_1], [x_0_3_0, x_0_3_1]], [[x_1_0_0, x_1_0_1], [x_1_1_0, x_1_1_1], [x_1_2_0, x_1_2_1], [x_1_3_0, x_1_3_1]], [[x_2_0_0, x_2_0_1], [x_2_1_0, x_2_1_1], [x_2_2_0, x_2_2_1], [x_2_3_0, x_2_3_1]], [[x_3_0_0, x_3_0_1], [x_3_1_0, x_3_1_1], [x_3_2_0, x_3_2_1], [x_3_3_0, x_3_3_1]]]


In [28]:
constraints = []

for i in range(n + 1):
    # main diagonal equal to 0 (no self loops)
    constraints.append(PbEq([(graph_nodes[i][i][k], 1) for k in range(m)], 0))

# Each node visited only once
for i in range(n):
    constraints.append(And([PbEq([(graph_nodes[i][j][k],1) for k in range(m) for j in range(n+1)
                            ],1),
                            PbEq([(graph_nodes[j][i][k],1) for k in range(m) for j in range(n+1)
                            ],1)]))
    
# Each courier can go to the origin at maximum once
for k in range(m):
    constraints.append(And([PbEq([(graph_nodes[n][j][k],1) for j in range(n)
                    ],1),
                    PbEq([(graph_nodes[j][n][k],1) for j in range(n)
                    ],1)]))

# the load of each courier cannot exceed its load size
for k in range(m):
    for i in range(n):
        constraints.append(PbLe([(graph_nodes[i][j][k],item_size[i]) for i in range(n) for j in range(n+1)], load_size[k]))

# n arcs in n arcs out
for k in range(m):
    for j in range(n+1):
        constraints.append(Sum([If(graph_nodes[i][j][k], 1, 0) for i in range(n+1)]) == Sum([If(graph_nodes[j][i][k], 1, 0) for i in range(n+1)]))

# DFJ subtour elimination: The number of arcs between nodes in the subset should be less than the number of nodes in that subset.
for k in range(m):
    for i in range(n):
        for j in range(n):
            constraints.append(Implies(And(graph_nodes[i][j][k], graph_nodes[j][i][k]), PbLe([(graph_nodes[i][j][k],1) for i in range(n) for j in range(n+1)], n)))


# total distance
# total_distance = Int("total_distance")
# have to sum over all k but also need to add the distance from the depot to the first item and from the last item to the depot
# constraints.append(total_distance == Sum([If(graph_nodes[i][j][k], distance[i][j].item(), 0) for i in range(n + 1) for j in range(n + 1) for k in range(m)]))
max_distance_per_courier = [Int("max_distance_per_courier_%s" % k) for k in range(m)]
for k in range(m):
    constraints.append(max_distance_per_courier[k] == Sum([If(graph_nodes[i][j][k], distance[i][j].item(), 0) for i in range(n + 1) for j in range(n + 1)]))

best_max_distance = math.inf
graph_node_per_courier = [[] for k in range(m)]
s = Solver()
s.add(constraints)
for l in range(1000):
    if s.check()==sat:
        model = s.model()
        max_distance = max([model[max_distance_per_courier[k]].as_long() for k in range(m)])
        if max_distance < best_max_distance:
            best_max_distance = max_distance
            graph_node_per_courier = [[] for k in range(m)]
            for k in range(m):
                s.add(max_distance_per_courier[k] < max_distance)
                for i in range(n + 1):
                    for j in range(n + 1):
                        if model[graph_nodes[i][j][k]] == True:
                            graph_node_per_courier[k].append((i,j))
         # print("unsat")
print("best_max_distance:", best_max_distance)
print("graph_node_per_courier:", graph_node_per_courier)

# order the path of each courier of the variable graph_node_per_courier
paths = [[] for k in range(m)]
for k in range(m):
    paths[k].append(n)
    count = len(graph_node_per_courier[k])-1
    for i in range(count):
        for elem in graph_node_per_courier[k]:
            if elem[0] == paths[k][-1]:
                paths[k].append(elem[1])
                break
    paths[k].append(n)
# remove from paths first and last element
for k in range(m):
    paths[k].pop(0)
    paths[k].pop(-1)
print("paths:", paths)
#s.minimize(total_distance)
#if s.check() == sat:
 #   model = s.model()

 #   print("total_distance:", model[total_distance])
 #   print("graph_nodes:")
 #   # print the human readable solution
 #   for k in range(m):
 #       print("courier", k)
 #       for i in range(n + 1):
 #           for j in range(n + 1):
 #               if model[graph_nodes[i][j][k]] == True:
 #                   print(i, "->", j)
    # plot the solution
# else:
  #  print("unsat")


best_max_distance: 206
graph_node_per_courier: [[(1, 3), (3, 1)], [(0, 2), (2, 3), (3, 0)]]
paths: [[1], [0, 2]]
